<h2>Team Members</h2>
<ul>
<li>Sohad Hossam Eldin 1190019</li>
<li>Bassant Hisham Mohamed 1190018</li>
<li>Yasmin Hashem Niazy 4200013</li>
<li>Mary Ashraf 1190322</li>
</ul>

In [1]:
#%pip install pyarabic 

In [2]:
#%pip install nltk

In [3]:
#%pip install --upgrade pip


In [4]:
#%pip install diacritization-evaluation

In [5]:
#%pip install gensim


In [6]:
#%pip install scikit-learn


In [7]:
#%pip install torch


In [8]:
#%pip install tensorflow


In [9]:
#%pip install keras

In [10]:
#%pip install --upgrade tensorflow


In [11]:
#%pip install --upgrade keras

In [21]:
%pip install --upgrade pip


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 1.3 MB/s eta 0:00:02
   -- ------------------------------------- 0.1/2.1 MB 1.3 MB/s eta 0:00:02
   ---- ----------------------------------- 0.2/2.1 MB 1.7 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.1 MB 1.7 MB/s eta 0:00:02
   ----------- ---------------------------- 0.6/2.1 MB 2.7 MB/s eta 0:00:01
   -------------- ------------------------- 0.8/2.1 MB 2.8 MB/s eta 0:00:01
   ------------------ --------------------- 1.0/2.1 MB 3.0 MB/s eta 0:00:01
   --------------------- ------------------ 1.1/2.1 MB 3.1 MB/s eta 0:00:01
   ------------------------- -------------- 1.3/2.1 MB 3.2 MB/s eta 0:00:01
   ---------------------------- ----------- 1.5/2.1 MB 3.3 MB/s eta 0:00:01
   -------------------------------- ------- 1.7/2.1 MB 3.4 MB/s eta 0:00:01
   ----------------------------------- ---- 1.9/2.1 MB 3.4 MB/s eta 0:00:01
   ----------------

In [23]:
%pip uninstall tensorflow
%pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


  Using cached tensorflow-2.15.0-cp310-cp310-win_amd64.whl.metadata (3.6 kB)
  Using cached tensorflow_intel-2.15.0-cp310-cp310-win_amd64.whl.metadata (5.1 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached tensorflow-2.15.0-cp310-cp310-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.15.0-cp310-cp310-win_amd64.whl (300.9 MB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.0.2
    Uninstalling keras-3.0.2:
      Successfully uninstalled keras-3.0.2
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\Lenovo\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



<h4>Imports</h4>

In [24]:


import re
from pyarabic.araby import strip_tashkeel
import nltk 
from nltk import word_tokenize
from diacritization_evaluation import util
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from gensim.models import Word2Vec

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense


nltk.download('punkt')

ModuleNotFoundError: No module named 'tensorflow.python'

<h3>Dataset class<h3>

In [ ]:

class CSVDataset(Dataset):

    def __init__(self, path):

        dataset_txt = open(path, "r", encoding='utf-8').read()
        #print(type(dataset_txt))

        chars_to_remove = r"\(\s*[a-zA-Z0-9_-]+\s*/\s*[a-zA-Z0-9_-]+\s*\)|[a-zA-Z0-9_-]+" 
        dataset_cleaned = re.sub(chars_to_remove, "", dataset_txt)
        dataset_sentences = re.split(r"\s*\.\s*|\n|\s*،\s*|\s*:\s*|\s*[()]\s*|\s*؛\s*|\s*؟\s*|\s*!\s*|\s*\"\s*",dataset_cleaned)

        self.x_train_letters,y_train_letters,self.x_train_words,y_train_words= self.Tokenizing(dataset_sentences)

        self.y_train_letters_numbered = LabelEncoder().fit_transform(y_train_letters)
        self.y_train_words_numbered = LabelEncoder().fit_transform(y_train_words)

        #dictionary of the labels(words,letters) before converted to numbers
        self.encoding_mapping_y_letters = {encoded_label: original_label for encoded_label, original_label  in zip(self.y_train_letters_numbered, y_train_letters)}
        self.encoding_mapping_y_words = {encoded_label: original_label for encoded_label, original_label  in zip( self.y_train_words_numbered, y_train_words)}  

        print(self.y_train_letters_numbered.shape)
        print(self.y_train_words_numbered.shape)
        print( self.y_train_letters_numbered[:10])
        print( self.y_train_words_numbered[:10])

        self.y_train_letters_numbered = self.y_train_letters_numbered.reshape((len(self.y_train_letters_numbered), 1))
        self.y_train_words_numbered = self.y_train_words_numbered.reshape((len(self.y_train_words_numbered), 1))

        print(self.y_train_letters_numbered.shape)
        print(self.y_train_words_numbered.shape)
        print( self.y_train_letters_numbered[:10])
        print( self.y_train_words_numbered[:10])


    def Tokenizing(self,dataset_sentences):

        x_train_letters = []
        y_train_letters = []
        x_train_words = []
        y_train_words = []
        for sentence in dataset_sentences:
            sentence_wihout_diacratics = strip_tashkeel(sentence)
            tokens = word_tokenize(sentence, language="arabic", preserve_line=True)
            tokens_wihtout_diacratics = word_tokenize(sentence_wihout_diacratics, language="arabic", preserve_line=True)

            for word in tokens:
                text, inputs, diacritics =util.extract_haraqat(word)
                inputs.insert(0, "<s>")
                inputs.append("</s>")

                diacritics.insert(0, "<s>")
                diacritics.append("</s>")
                
                x_train_letters.extend(inputs)
                y_train_letters.extend(diacritics)


            if(len(tokens)) :
                tokens.insert(0, "<s>")
                tokens.append("</s>")
                y_train_words.extend(tokens)
            if(len(tokens_wihtout_diacratics)) :
                tokens_wihtout_diacratics.insert(0, "<s>")
                tokens_wihtout_diacratics.append("</s>")
                x_train_words.extend(tokens_wihtout_diacratics)

        
        print(x_train_letters[0:10])
        print(y_train_letters[0:10])
        print(y_train_words[0:10])
        print(x_train_words[0:10])
        return x_train_letters,y_train_letters,x_train_words,y_train_words

    def __len__(self):
        return len(self.X)

    # get a row at an index
    # The __getitem__ function loads and returns a sample from the dataset at the given index idx
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])


# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    # The Dataset retrieves our dataset’s features and labels one sample at a time.
    # While training a model, we typically want to pass samples in “minibatches”,
    # reshuffle the data at every epoch to reduce model overfitting,
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return dataset.encoding_mapping, train_dl, test_dl

<h4>Reading the dataset</h4>

In [ ]:
dataset_txt = open(r"train.txt", "r", encoding='utf-8').read()
print(type(dataset_txt))
#print(dataset_txt)

<class 'str'>


<h4>Cleaning the dataset</h4>

In [ ]:
# do we need to remove [ :ص]
chars_to_remove = r"\(\s*[a-zA-Z0-9_-]+\s*/\s*[a-zA-Z0-9_-]+\s*\)|[a-zA-Z0-9_-]+" 
dataset_cleaned = re.sub(chars_to_remove, "", dataset_txt)
#print(dataset_cleaned)

<h4>Segmenting dataset into sentences</h4>

In [ ]:
# r'(\([^)]*\))' should we remove the brackets or not
dataset_sentences = re.split(r"\s*\.\s*|\n|\s*،\s*|\s*:\s*|\s*[()]\s*|\s*؛\s*|\s*؟\s*|\s*!\s*|\s*\"\s*",dataset_cleaned)
print(type(dataset_sentences))
print(dataset_sentences[0:30])

<class 'list'>
['قَوْلُهُ', '', 'أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ', 'قَالَ الزَّرْكَشِيُّ', 'ابْنُ عَرَفَةَ', 'قَوْلُهُ', 'بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً', 'كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ', 'ابْنُ عَرَفَةَ', 'قَوْلُ ابْنِ شَاسٍ', 'أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ', 'وَسِحْرٍ', 'مُحَمَّدٌ', 'قَوْلُ مَالِكٍ وَأَصْحَابِهِ أَنَّ السَّاحِرَ كَافِرٌ بِاَللَّهِ تَعَالَى قَالَ مَالِكٌ', 'هُوَ كَالزِّنْدِيقِ إذَا عَمِلَ السِّحْرَ بِنَفْسِهِ قُتِلَ وَلَمْ يُسْتَتَبْ', '', 'قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ', 'أَيْ الْوَصِيَّةُ', 'قَوْلُهُ مَا مَرَّ', 'أَيْ قُبَيْلَ قَوْلِ الْمَتْنِ لَغَتْ وَلَوْ اقْتَصَرَ عَلَى أَوْصَيْت لَهُ بِشَاةٍ أَوْ أَعْطُوهُ شَاةً وَلَا غَنَمَ لَهُ عِنْدَ الْمَوْتِ هَلْ تَبْطُلُ الْوَصِيَّةُ أَوْ يُشْتَرَى لَهُ شَاةٌ وَيُؤْخَذُ مِنْ قَوْلِهِ الْآتِي كَمَا ل

<h4>Tokenizing the sentence</h4>

In [ ]:
x_train_letters = []
y_train_letters = []
x_train_words = []
y_train_words = []
for sentence in dataset_sentences:
    sentence_wihout_diacratics = strip_tashkeel(sentence)
    tokens = word_tokenize(sentence, language="arabic", preserve_line=True)
    tokens_wihtout_diacratics = word_tokenize(sentence_wihout_diacratics, language="arabic", preserve_line=True)
   
    for word in tokens:
        text, inputs, diacritics =util.extract_haraqat(word)
        x_train_letters.append(inputs)
        y_train_letters.append(diacritics)

    if(len(tokens)) :
        tokens.insert(0, "<s>")
        tokens.append("</s>")
        y_train_words.append(tokens)
    if(len(tokens_wihtout_diacratics)) :
        tokens_wihtout_diacratics.insert(0, "<s>")
        tokens_wihtout_diacratics.append("</s>")
        x_train_words.append(tokens_wihtout_diacratics)

## tokenizing the words

In [ ]:
print(x_train_letters[0:10])
print(y_train_letters[0:10])
print(y_train_words[0:10])
print(x_train_words[0:10])

[['ق', 'و', 'ل', 'ه'], ['أ', 'و'], ['ق', 'ط', 'ع'], ['ا', 'ل', 'أ', 'و', 'ل'], ['ي', 'د', 'ه'], ['إ', 'ل', 'خ'], ['ق', 'ا', 'ل'], ['ا', 'ل', 'ز', 'ر', 'ك', 'ش', 'ي'], ['ا', 'ب', 'ن'], ['ع', 'ر', 'ف', 'ة']]
[['َ', 'ْ', 'ُ', 'ُ'], ['َ', 'ْ'], ['َ', 'َ', 'َ'], ['', 'ْ', 'َ', 'َّ', 'ُ'], ['َ', 'َ', 'ُ'], ['', 'َ', 'ْ'], ['َ', '', 'َ'], ['', '', 'َّ', 'ْ', 'َ', 'ِ', 'ُّ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ']]
[['<s>', 'قَوْلُهُ', '</s>'], ['<s>', 'أَوْ', 'قَطَعَ', 'الْأَوَّلُ', 'يَدَهُ', 'إلَخْ', '</s>'], ['<s>', 'قَالَ', 'الزَّرْكَشِيُّ', '</s>'], ['<s>', 'ابْنُ', 'عَرَفَةَ', '</s>'], ['<s>', 'قَوْلُهُ', '</s>'], ['<s>', 'بِلَفْظٍ', 'يَقْتَضِيه', 'كَإِنْكَارِ', 'غَيْرِ', 'حَدِيثٍ', 'بِالْإِسْلَامِ', 'وُجُوبَ', 'مَا', 'عُلِمَ', 'وُجُوبُهُ', 'مِنْ', 'الدِّينِ', 'ضَرُورَةً', '</s>'], ['<s>', 'كَإِلْقَاءِ', 'مُصْحَفٍ', 'بِقَذَرٍ', 'وَشَدِّ', 'زُنَّارٍ', '</s>'], ['<s>', 'ابْنُ', 'عَرَفَةَ', '</s>'], ['<s>', 'قَوْلُ', 'ابْنِ', 'شَاسٍ', '</s>'], ['<s>', 'أَوْ', 'بِفِعْلٍ', 'يَتَضَمَّنُهُ', 'هُو

In [ ]:
print(type(x_train_letters))
print(x_train_letters[2])

#ouh lala , tokinze is dividing the tashkelat as indpendent letters as well , what does that tell us tho 

<class 'list'>
['ق', 'ط', 'ع']


In [ ]:
temp=CSVDataset(r"train.txt") 

['<s>', 'ق', 'و', 'ل', 'ه', '</s>', '<s>', 'أ', 'و', '</s>']
['<s>', 'َ', 'ْ', 'ُ', 'ُ', '</s>', '<s>', 'َ', 'ْ', '</s>']
['<s>', 'قَوْلُهُ', '</s>', '<s>', 'أَوْ', 'قَطَعَ', 'الْأَوَّلُ', 'يَدَهُ', 'إلَخْ', '</s>']
['<s>', 'قوله', '</s>', '<s>', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', '</s>']
(12590722,)
(2658021,)
[ 2  6 16  7  7  1  2  6 16  1]
[     6  91505      5      6   6306  91125  21191 154522   9139      5]
(12590722, 1)
(2658021, 1)
[[ 2]
 [ 6]
 [16]
 [ 7]
 [ 7]
 [ 1]
 [ 2]
 [ 6]
 [16]
 [ 1]]
[[     6]
 [ 91505]
 [     5]
 [     6]
 [  6306]
 [ 91125]
 [ 21191]
 [154522]
 [  9139]
 [     5]]


# Word2vec Feature Extraction

In [13]:
word2vec_model = Word2Vec(
    x_train_words,  # list of sentences, if you don't have all the data in RAM, you can give a file name to corpus_file
    vector_size=50,  # output size of word embedding
    window=4,  # window size
    min_count=1,  # ignores all the words with total frequency lower than this
    workers=5,  # number of workers to use
    sg=1,  # skip-gram
    hs=0,  # 1 --> hierarchical, 0 --> negative sampling
    negative=5,  # how many negative samples
    alpha=0.03,  # the initial learning rate
    min_alpha=0.0001,  # learning rate will linearly drop to min_alpha as training progresses
    seed=54,  # random seed
    iter=10,
    compute_loss=True,  # number of iterations
)


NameError: name 'x_train_words' is not defined